# Pandas: Transforming and Aggregating data

(c) Wouter van Atteveldt, CC-BY-SA

# Outline

1. <b>Restructuring data
   + Changing values: map and replace
   + Binning and dichotomizing
   + String values</b>
1. Reshaping data
   + Pivoting data
   + Hierarchical indices
   + Stack and unstack
2. Aggregating data
   + Split - Apply - Combine
   + Pivot tables
4. Lab 4: Transforming Movie Ratings

# Changing values

+ `map` to apply a mapping to a column
    + `column.map(dict/function)`
+ `replace` to replace value(s)
+ `pd.cut` and `pd.dcut` to discretize (bin) a value

# Restructuring data

Let's get some data to work with:

In [1]:
import pandas as pd
d = pd.DataFrame({"Name": ["John", "Sue", "Mary"], "Age": [23, 19, 21], "Group": ["A", "A", "B"]})
d.head()


,Age,Group,Name
0,23,A,John
1,19,A,Sue
2,21,B,Mary


# Mapping values 

In [2]:
# Use a dict to replace values
d.Group.map({"A": 1, "B": 2})

0    1
1    1
2    2
Name: Group, dtype: int64

In [3]:
# Can also use a series:
length = pd.Series({"John": 176, "Sue": 165, "Mary": 179})
d.Name.map(length)

0    176
1    165
2    179
Name: Name, dtype: int64

# Mapping with a function

In [4]:
def double(x):
    return x * 2
d.Age.map(double)

0    46
1    38
2    42
Name: Age, dtype: int64

In [5]:
# Using lambda expression
d.Age.map(lambda x:x * 2)

0    46
1    38
2    42
Name: Age, dtype: int64

# Replacing values

In [6]:
d.Name.replace("Sue", "Susan")

0     John
1    Susan
2     Mary
Name: Name, dtype: object

In [7]:
d.Name.replace(["Sue", "Mary"], "Susan")

0     John
1    Susan
2    Susan
Name: Name, dtype: object

In [8]:
d.Name.replace(["Sue", "Mary"], ["Susan", "Marianne"])

0        John
1       Susan
2    Marianne
Name: Name, dtype: object

# Discretization and binning

+ Change a continuous variable into groups
    + Groups often called 'bins'
+ Useful for machine learning, data summarization

# cut: Binning into equal-width bins

In [9]:
pd.cut(d.Age, [0, 20, 100])

0    (20, 100]
1      (0, 20]
2    (20, 100]
Name: Age, dtype: category
Categories (2, object): [(0, 20] < (20, 100]]

In [10]:
pd.cut(d.Age, [0, 20, 100], labels=["Young", "Old"])

0      Old
1    Young
2      Old
Name: Age, dtype: category
Categories (2, object): [Young < Old]

# Binning data

In [11]:
import numpy as np
data = pd.Series(np.random.randint(100, size=30))
data.head()

0    30
1    86
2    35
3    85
4    51
dtype: int64

In [12]:
pd.cut(data, 3, precision=1).head()

0    (2.9, 33]
1     (63, 93]
2     (33, 63]
3     (63, 93]
4     (33, 63]
dtype: category
Categories (3, object): [(2.9, 33] < (33, 63] < (63, 93]]

In [13]:
pd.cut(data, 3, labels=["lo", "mid", "hi"]).head()

0     lo
1     hi
2    mid
3     hi
4    mid
dtype: category
Categories (3, object): [lo < mid < hi]

In [14]:
pd.cut(data, 3, labels=["lo", "mid", "hi"]).value_counts()

lo     12
mid    10
hi      8
dtype: int64

# Binning into equal sized bins

In [15]:
pd.qcut(data, 3).head()

0    (29, 59]
1    (59, 93]
2    (29, 59]
3    (59, 93]
4    (29, 59]
dtype: category
Categories (3, object): [[3, 29] < (29, 59] < (59, 93]]

In [16]:
pd.qcut(data, 3).value_counts()

(59, 93]    10
(29, 59]    10
[3, 29]     10
dtype: int64

# Finding duplicates

In [17]:
dupes = pd.concat([d, d[0:1]])
dupes.head()

,Age,Group,Name
0,23,A,John
1,19,A,Sue
2,21,B,Mary
0,23,A,John


In [18]:
dupes.duplicated()

0    False
1    False
2    False
0     True
dtype: bool

# Dropping duplicates 

In [19]:
dupes.drop_duplicates()

,Age,Group,Name
0,23,A,John
1,19,A,Sue
2,21,B,Mary


In [20]:
# Specify column to find duplicates and keep first, last, or False (none)
d.drop_duplicates("Group", keep="last")

,Age,Group,Name
1,19,A,Sue
2,21,B,Mary


In [21]:
d.drop_duplicates("Group", keep=False)

,Age,Group,Name
2,21,B,Mary


# Applying string functions

+ Can apply python lower, upper etc 
+ More efficient to use vectorized versions
+ column.str.function

In [22]:
d.Name.str.upper()

0    JOHN
1     SUE
2    MARY
Name: Name, dtype: object

In [23]:
d.Name.str.contains("e|y") 

0    False
1     True
2     True
Name: Name, dtype: bool

# Outline

1. Restructuring data
   + Changing values: map and replace
   + Binning and dichotomizing
   + String values
1. <b>Reshaping data
   + Hierarchical indices
   + Stack and unstack
   + Pivoting data</b>
2. Aggregating data
   + Split - Apply - Combine
   + Pivot tables
4. Lab 5: Transforming Movie Ratings

# Hierarchical indexes

+ Rows (and columns) can have nested indices
    + e.g. reviews within products
+ Can use these indices to select groups of rows
+ Can 'stack' long data to wide, or 'unstack' wide to long 

In [24]:
# Hierarchical index

In [25]:
d = d.set_index(["Group", "Name"])
d

Age
Group Name     
A     John   23
      Sue    19
B     Mary   21

In [26]:
d.ix["A"]

,Age
Name,
John,23
Sue,19


# Reordering nested indices

In [27]:
d2 = pd.DataFrame([["John", 1, 12, 16], ["John", 2, 15, 19], 
                  ["Mary", 1, 14, 12], ["Mary", 2, 11, 9]],
                 columns=["Name", "Trial", "x", "y"])
d2 = d2.set_index(['Name', 'Trial'])
d2

x   y
Name Trial        
John 1      12  16
     2      15  19
Mary 1      14  12
     2      11   9

In [28]:
d2.swaplevel("Trial", "Name").ix[1]

,x,y
Name,,
John,12,16
Mary,14,12


# Stacking and unstacking

+ Long form is also called 'stacked' form
+ `stack` and `unstack` move between long and wide
+ Can specify which index to (un)stack

In [29]:
d2.stack()

Name  Trial   
John  1      x    12
             y    16
      2      x    15
             y    19
Mary  1      x    14
             y    12
      2      x    11
             y     9
dtype: int64

In [30]:
d2.unstack()

x       y    
Trial   1   2   1   2
Name                 
John   12  15  16  19
Mary   14  11  12   9

# Combine unstack and stack to swap rows and columns

In [31]:
d2

x   y
Name Trial        
John 1      12  16
     2      15  19
Mary 1      14  12
     2      11   9

In [32]:
d2.unstack("Name").stack(0)

Name     John  Mary
Trial              
1     x    12    14
      y    16    12
2     x    15    11
      y    19     9

# Pivoting

+ Stack/unstack assumes hierarchical indices
+ Pivot offers stacking (reshaping) from normal columns

In [33]:
x = d2.reset_index()
x.head()

,Name,Trial,x,y
0,John,1,12,16
1,John,2,15,19
2,Mary,1,14,12
3,Mary,2,11,9


In [34]:
x.pivot("Name", "Trial", "x")

Trial,1,2
Name,,
John,12,15
Mary,14,11


In [35]:
x.pivot("Name", "Trial")

x       y    
Trial   1   2   1   2
Name                 
John   12  15  16  19
Mary   14  11  12   9

# Outline

1. Restructuring data
   + Changing values: map and replace
   + Binning and dichotomizing
   + String values
1. Reshaping data
   + Pivoting data
   + Hierarchical indices
   + Stackc and unstack
2. <b>Aggregating data
   + Split - Apply - Combine
   + Pivot tables
   + Group operations</b>
4. Lab 5: Transforming Movie Ratings

# Aggregating data

+ Compute *summary* of *values* per *group*
    + e.g. average rating per genre
+ Split data with `groupby`
+ Apply built-in or custom aggregate function
+ Result is combined data frame

![SAC](http://i.imgur.com/MaAPZAV.png)

# Grouping by column

In [36]:
x

,Name,Trial,x,y
0,John,1,12,16
1,John,2,15,19
2,Mary,1,14,12
3,Mary,2,11,9


In [37]:
x.groupby("Name")

In [38]:
x.groupby("Name").mean()

,Trial,x,y
Name,,,
John,1.5,13.5,17.5
Mary,1.5,12.5,10.5


In [39]:
x.groupby("Name").x.mean()

Name
John    13.5
Mary    12.5
Name: x, dtype: float64

# Group by index

In [40]:
d2

x   y
Name Trial        
John 1      12  16
     2      15  19
Mary 1      14  12
     2      11   9

In [41]:
d2.groupby(level="Trial").mean()

,x,y
Trial,,
1,13,14
2,13,14


# Group by multiple values

In [42]:
x.groupby(["Name", "Trial"]).mean()

x   y
Name Trial        
John 1      12  16
     2      15  19
Mary 1      14  12
     2      11   9

# Data Aggregation

+ Compute summary for multiple values
+ Can use built-in functions (count, mean, ..)
+ Can use custom functions via `.agg`
    + Can be much slower than built-in functions
+ Apply to all columns or to a single column
    + `x.groupby(..)[columns].mean()`

# Data Aggregation

In [43]:
# apply function directly or with agg function
x.groupby(["Name"]).mean()
x.groupby(["Name"]).agg('mean')

,Trial,x,y
Name,,,
John,1.5,13.5,17.5
Mary,1.5,12.5,10.5


In [44]:
# Can use custom function
x.groupby(["Name"]).agg(lambda x: sum(x) / len(x))

,Trial,x,y
Name,,,
John,1.5,13.5,17.5
Mary,1.5,12.5,10.5


In [45]:
%timeit -n 10 x.groupby(["Name"]).agg(lambda x: sum(x) / len(x))

10 loops, best of 3: 4.53 ms per loop


In [46]:
%timeit -n 10 x.groupby(["Name"]).mean

10 loops, best of 3: 48.9 µs per loop


# Computing multiple summary functions using agg

+ `.agg` can apply one or more functions
+ Can specify functions per column using dict
+ Can apply python functions (pass by reference, e.g. `max` or `my_function`)
+ Or use built-in vectorized functions (pass by name, e.g. `'max'`)

In [47]:
x.groupby("Name")["x"].agg(['mean', 'min', 'max'])

,mean,min,max
Name,,,
John,13.5,12,15
Mary,12.5,11,14


In [48]:
x.groupby("Name").agg({"x": "mean", "y": ["min", "max"]})

y         x
     min max  mean
Name              
John  16  19  13.5
Mary   9  12  12.5

# Group-wise transformations

+ Aggregate functions return one value per group
+ Can also return value for each member of a group
    + e.g. centering (demeaning): substract mean from each element
    + use `groups.transform(function)`
+ Can even return arbitrary number of values for each group
    + e.g. get top-5 ranked items per category
    + use `groups.apply(function)

In [49]:
def center(values):
    return values - values.mean()
x.groupby("Name").transform(center)

,Trial,x,y
0,-0.5,-1.5,-1.5
1,0.5,1.5,1.5
2,-0.5,1.5,1.5
3,0.5,-1.5,-1.5


# Apply: Computing top-n values

In [50]:
extra = pd.DataFrame({"Name": ["John", "Mary"], "Trial": [3,3], "x": [11,17]})
x = x.append(extra).sort_values("Name")
x

,Name,Trial,x,y
0,John,1,12,16.0
1,John,2,15,19.0
0,John,3,11,NaN
2,Mary,1,14,12.0
3,Mary,2,11,9.0
1,Mary,3,17,NaN


In [51]:
def top(df, n=2, column='x'):
    return df.sort_values(by=column, ascending=False).head(n)
top(x)

,Name,Trial,x,y
1,Mary,3,17,NaN
1,John,2,15,19.0


In [52]:
x.groupby("Name").apply(top)

Name  Trial   x     y
Name                         
John 1  John      2  15  19.0
     0  John      1  12  16.0
Mary 1  Mary      3  17   NaN
     2  Mary      1  14  12.0

In [53]:
# Can supply extra arguments
x.groupby("Name").apply(top, n=1, column='y')

,,Name,Trial,x,y
Name,,,,,
John,1,John,2,15,19.0
Mary,2,Mary,1,14,12.0


# Example: imputing missing values

In [54]:
x.y

0    16.0
1    19.0
0     NaN
2    12.0
3     9.0
1     NaN
Name: y, dtype: float64

In [55]:
def impute_mean(values):
    return values.fillna(values.mean())
impute_mean(x.y)

0    16.0
1    19.0
0    14.0
2    12.0
3     9.0
1    14.0
Name: y, dtype: float64

In [56]:
x.groupby("Name").apply(impute_mean)

Name  Trial   x     y
Name                         
John 0  John      1  12  16.0
     1  John      2  15  19.0
     0  John      3  11  17.5
Mary 2  Mary      1  14  12.0
     3  Mary      2  11   9.0
     1  Mary      3  17  10.5

# Pivot tables and crosstabs

+ `pivot_table`: Shortcut for groupby + aggregate with useful defaults:
    + default uses mean function
    + default uses all numeric columns for values
+ `crosstab`: Shortcut for groupby with counts
    + calculate occurences by one or more groups

# Pivot Tables 

In [57]:
x.pivot_table(index=["Name", "Trial"])

x     y
Name Trial          
John 1      12  16.0
     2      15  19.0
     3      11   NaN
Mary 1      14  12.0
     2      11   9.0
     3      17   NaN

In [58]:
x.pivot_table(index="Name", columns="Trial")

x             y          
Trial   1   2   3     1     2   3
Name                             
John   12  15  11  16.0  19.0 NaN
Mary   14  11  17  12.0   9.0 NaN

In [59]:
x.pivot_table(index="Name", values="x", aggfunc='sum')

Name
John    38
Mary    42
Name: x, dtype: int64

# Crosstabs

In [60]:
pd.crosstab(x.Name, x.Trial, margins=True)

Trial,1,2,3,All
Name,,,,
John,1,1,1,3
Mary,1,1,1,3
All,2,2,2,6


# Outline

1. Restructuring data
   + Changing values: map and replace
   + Binning and dichotomizing
   + String values
1. Reshaping data
   + Pivoting data
   + Hierarchical indices
   + Stackc and unstack
2. Aggregating data
   + Split - Apply - Combine
   + Pivot tables
   + Group operations
4. <b>Lab 4: Transforming Movie Ratings</b>

# Lab 4: Transforming Movie Ratings

+ Get the movielens data set
   + Ratings, tags, genres per movies
+ Pick research question that uses tags and/or genres, e.g.
   + most-used tags per genre
   + correlation between rating and number of tags
+ Use transformation and aggregation to answer the research question 